In [ ]:
import praw
from itertools import islice
import spacy


In [ ]:
nlp = spacy.load("en_core_web_sm")

reddit = praw.Reddit(
    client_id="7fvE1QgAPTvqElyf0l5l0A",
    client_secret="uK0T064WyyBpOfmu79KnoE7depoE_Q",
    user_agent="char-rnn-AMA by u/Axel"
)

subreddit = reddit.subreddit("neutralnews")

dataset = []

# Customize your range
start = 0
end = 10000

suffixes = ("an", "ese", "ian", "ish", "i", "ic")

# top, hot, new, rising, controversial, 
for post in islice(subreddit.controversial(limit = None), start, end):
    if not post.stickied and post.selftext != '[deleted]':
        title = post.title.strip()
        doc = nlp(title)

        named_entities = [ent.text for ent in doc.ents if ent.label_ in {"PERSON", "ORG", "GPE", "LOC", "EVENT"} and len(ent.text.split()) <= 3]

        nationality_adjectives = [
            token.text for token in doc
            if token.pos_ == "ADJ" and token.text.lower().endswith(suffixes) and token.is_alpha
        ]

        all_entities = list(set(named_entities + nationality_adjectives))

        dataset.append((all_entities, title))

print(f"Collected {len(dataset)} entries with NER.")

with open("neutralnews.txt", "a", encoding="utf-8") as f:
    for entities, title in dataset:
        entity_str = ", ".join(entities)
        f.write(f"({entity_str}) ||| {title}\n")


Collected 997 entries with NER.
